<H1>PRIORI-T</H1>
<p></p>

## Data taken from

Data was taken from the supplemental material of [Rao A, et al. (2020) PRIORI-T: A tool for rare disease gene prioritization using MEDLINE. PLoS One 15:e0231728](https://pubmed.ncbi.nlm.nih.gov/32315351/).

PRIORI-T was used for rare disease gene prioritization on a dataset of 230 solved rare disease real-world clinical cases compiled from four recent publications 

- Stavropoulos DJ, et al. (2016) Whole Genome Sequencing Expands Diagnostic Utility and Improves Clinical Management in Pediatric Medicine. NPJ Genom Med. 1:15012 PMID:28567303
- Bone WP, et al (2016) Computational evaluation of exome sequence data using human and model organism phenotypes improves diagnostic efficiency. Genet Med 18:608-17. PMID:26562225
- Stelzer G, et al. (2016) VarElect: the phenotype-based variation prioritizer of the GeneCards Suite. BMC Genomics. 17 Suppl 2:444.  PMID:27357693
- Lee H, et al. (2014) Clinical exome sequencing for genetic identification of rare Mendelian disorders. JAMA 312:1880-7.  PMID:25326637

## Probands
Some probands were excluded because it was npot possible to determine the specific (genetic) diagnosis from the available input data or because the diagnosis was not unambiguous. Cases with mixed diagnoses were excluded because cosegregation data was not presented that would allow an unambiguous diagnosis
- TCS021 (mixed phenotype)
- TCS022 (unclear disease-level diagnosis)
- TCS024 (mixed phenotype)
- TCS025 (mixed phenotype)
- TCS041 (Systemic capillary leak syndrome and TLN1 association not confirmed)
- TCS051
- TCS053
- TCS056

In [8]:
import phenopackets as php
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.protobuf.json_format import Parse, ParseDict
import pandas as pd
import math
from csv import DictReader
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
import re
import pyphetools
from pyphetools.creation import *
from pyphetools.visualization import PhenopacketTable
print(f"pyphetools version {pyphetools.__version__}")


pyphetools version 0.8.3


In [9]:
df = pd.read_excel("input/Supplementary1_PRIORIT.xlsx")

In [10]:
df.head()

,Internal ID,Case ID,Sex,Summary Clinical Indications and Phenotype,Clinical Indication and Phenotype (HPO),Referal disease,Diagnosed Disorder,Orphanet ID\n(Diagnosed Disorder),Partial Diagnosis,Gene,HGNC Gene ID,(NM) or Locus and Genomic Variant (zygosity),Inheritance Pattern,Remarks,Unnamed: 14,Author,PMID
0,TCS001,1004.0,F,"Lipomyelomeningocele, microcephaly, developmental delay","HP:0010301, HP:0000252, HP:0001263",NaN,Rubinstein-Taybi Syndrome 2,ORPHA353284 (Rubinstein-Taybi syndrome due to EP300 haploinsufficiency),NaN,EP300,3373,NM_001429.3:c.5723dupC (p.Thr1909Asnfs*164) (het),AD,NaN,NaN,Stavropoulos,PMID:28567304
1,TCS003,1008.0,M,"Global Developmental Delay, Scoliosis, joint hypermobility, hypoplasia of the corpus callosum, facial dysmorphology","HP:0001263, HP:0002650, HP:0001382, HP:0002079, HP:0001999",Noonan-Costello,Coffin-Siris syndrome,ORPHA1465 (Coffin-Siris syndrome),NaN,SMARCB1,11103,NM_003073.3:c.364del (p.Glu122Asnfs*21) (het),AD,NaN,NaN,Stavropoulos,PMID:28567305
2,TCS004,1009.0,M,"Developmental Delay, Short Stature, Metaphyseal Dysplasia","HP:0001263, HP:0004322, HP:0100255",RASopathy,Alazami Syndrome,"ORPHA319671 (Microcephalic primordial dwarfism, Alazami type)",NaN,LARP7,24912,NM_016648.2:c.756_757del (p.Arg253Ile*6) (hom),AR,NaN,NaN,Stavropoulos,PMID:28567306
3,TCS005,1012.0,M,Neonatal encephalopathy,HP:0007105,NaN,KAT6B-Related Disorder,"ORPHA85201 (Genitopatellar syndrome), ORPHA3047 (Blepharophimosis-intellectual disability syndrome, SBBYS type) or some other KAT6B-related disorder",NaN,KAT6B,17582,NM_012330.3:c.3021+1G>C (p?) (het),AD,NaN,NaN,Stavropoulos,PMID:28567307
4,TCS006,1015.0,M,"Brachydactyly of the second digit, ulnar deviation of the fingers, single transverse palmar crease bilaterally, left talipes equinovarus","HP:0009372, HP:0001193, HP:0007598, HP:0001762",NaN,Type C Brachydactyly,ORPHA93384 (Brachydactyly type C),NaN,GDF5,4220,NM_000557.2:c.847G>A (p.Val283Met) (het),AD,NaN,NaN,Stavropoulos,PMID:28567308


In [14]:
dis = df[["Diagnosed Disorder", "Gene"]]
dis

,Diagnosed Disorder,Gene
0,Rubinstein-Taybi Syndrome 2,EP300
1,Coffin-Siris syndrome,SMARCB1
2,Alazami Syndrome,LARP7
3,KAT6B-Related Disorder,KAT6B
4,Type C Brachydactyly,GDF5
...,...,...
224,autosomal dominant pseudohypoparathyroidism types Ia (PHP Ia) [MIM: 103580]; Ic (PHP Ic) [MIM: 612462],GNAS
225,autosomal dominant SHORT syndrome [MIM: 269880],PIK3R1
226,"autosomal dominant sex-limited 46,XY sex reversal 3 [MIM:612965]",NR5A1
227,autosomal dominant retinitis pigmentosa (type 37) [MIM: 611131]; enhanced S-cone syndrome [MIM: 268100],NR2E3
